In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

# 0. Install and Import dependencies

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

/Users/krishan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing_styles = mp.solutions.drawing_styles  # Drawing styles

# 1. Make some detections   

In [136]:
cap = cv2.VideoCapture(0)  # 0 for webcam

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance by making image writeable to false as we are not going to edit the image 

        # Make detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Drawing face landmarks on the screen
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, results.face_landmarks, 
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, 
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
            
#         used this line of code to change the colour of connections in the right hand
#         mp_drawing.DrawingSpec(colour=(0,0,255),thickness=2,circle_radius=2)

        # Right Hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

        # Pose Detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1704940024.019140       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


# Capture Landmarks & Export to CSV
<img src="https://i.imgur.com/8bForKY.png">
<img src="https://i.imgur.com/AzKNp7A.png">

In [137]:
#sample landmark coordinates
results.face_landmarks.landmark[0].y 

0.5873932838439941

In [6]:
import numpy as np
import csv
import os

In [138]:
num_cords = 0

if results.face_landmarks:
    num_cords += len(results.face_landmarks.landmark)
if results.pose_landmarks:
    num_cords += len(results.pose_landmarks.landmark)
if results.left_hand_landmarks:
    num_cords += len(results.left_hand_landmarks.landmark)
if results.right_hand_landmarks:
    num_cords += len(results.right_hand_landmarks.landmark)
num_cords

522

In [139]:
landmarks = ['class']
for val in range(1, num_cords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [140]:
#Now we have class column and 501 columns for x,y,z and v coordinates
landmarks[0:10]

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3']

In [141]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [151]:
class_name = "Five"

In [152]:
cap = cv2.VideoCapture(0)  # 0 for webcam

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance by making image writeable to false as we are not going to edit the image 

        # Make detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Drawing face landmarks on the screen
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, results.face_landmarks, 
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, 
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
            
#         used this line of code to change the colour of connections in the right hand
#         mp_drawing.DrawingSpec(colour=(0,0,255),thickness=2,circle_radius=2)

        # Right Hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

        # Pose Detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
        # Export coordinates
        try:

            #extracting pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            #extracting face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            #concatenating both face and pose landmarks
            row = pose_row+face_row
            #adding class name to the row
            row.insert(0, class_name)

            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1704940322.467928       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


# 3. Train Custom Model Using Scikit-learn

## 3.1 Read in collected data and process

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [156]:
df = pd.read_csv('coords.csv')

In [162]:
df

,class,0.5199763774871826,0.803003191947937,-1.0043673515319824,0.9976217150688171,0.5543729066848755,0.7062968015670776,-1.013872742652893,0.9942453503608704,0.5754266977310181,...,-0.03909732773900032,0.0.465,0.6112842559814453,0.6971602439880371,-0.026634424924850464,0.0.466,0.6168917417526245,0.6858875751495361,-0.027495795860886574,0.0.467
0,Looking Down,0.535347,0.812029,-1.805548,0.997771,0.566254,0.711728,-1.815560,0.994722,0.585122,...,-0.037896,0.0,0.613394,0.694047,-0.025039,0.0,0.618835,0.684762,-0.026042,0.0
1,Looking Down,0.541345,0.815550,-1.876920,0.997903,0.571658,0.713589,-1.885020,0.995150,0.590000,...,-0.037391,0.0,0.613227,0.695193,-0.023941,0.0,0.618444,0.686624,-0.024854,0.0
2,Looking Down,0.544462,0.817506,-1.848631,0.998019,0.574586,0.714434,-1.859255,0.995533,0.592701,...,-0.037814,0.0,0.612991,0.695920,-0.024354,0.0,0.618241,0.687358,-0.025285,0.0
3,Looking Down,0.546885,0.818934,-1.938705,0.998111,0.577210,0.714866,-1.945508,0.995872,0.595267,...,-0.037221,0.0,0.612517,0.695590,-0.023382,0.0,0.617745,0.687368,-0.024279,0.0
4,Looking Down,0.548011,0.818996,-1.910009,0.998214,0.578623,0.714860,-1.911336,0.996187,0.596813,...,-0.036822,0.0,0.612838,0.696454,-0.022929,0.0,0.618081,0.688134,-0.023800,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,Five,0.487322,0.548606,-0.619436,0.999999,0.511866,0.482971,-0.560085,0.999995,0.524985,...,0.003682,0.0,0.538466,0.489367,0.021326,0.0,0.542884,0.483639,0.022066,0.0
781,Five,0.487333,0.548015,-0.618375,0.999998,0.511831,0.482319,-0.559165,0.999995,0.524953,...,0.003720,0.0,0.539079,0.489424,0.021623,0.0,0.543483,0.483628,0.022392,0.0
782,Five,0.487319,0.547546,-0.616995,0.999998,0.511780,0.481735,-0.558159,0.999994,0.524896,...,0.003940,0.0,0.539096,0.489115,0.021723,0.0,0.543516,0.483289,0.022496,0.0
783,Five,0.487021,0.547299,-0.606167,0.999998,0.511529,0.481414,-0.548308,0.999994,0.524663,...,0.003835,0.0,0.539375,0.489107,0.021734,0.0,0.543800,0.483222,0.022515,0.0


In [163]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## 3.2 Train Classification Model

In [165]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [166]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [167]:
list(pipelines.values())[1]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgeclassifier', RidgeClassifier())])

In [168]:
#training the model
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

/Users/krishan/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [169]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [170]:
fit_models['lr'].predict(X_test[0:1])

array(['Smiling'], dtype=object)

## 3.3 Evaluate and Serialize Model

In [171]:
from sklearn.metrics import accuracy_score
import pickle

In [172]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [173]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make Detections with Model

In [174]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [175]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for webcam

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance by making image writeable to false as we are not going to edit the image 

        # Make detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Drawing face landmarks on the screen
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, results.face_landmarks, 
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, 
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
            
#         used this line of code to change the colour of connections in the right hand
#         mp_drawing.DrawingSpec(colour=(0,0,255),thickness=2,circle_radius=2)

        # Right Hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

        # Pose Detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
        # Export coordinates
        try:

            #extracting pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            #extracting face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            #concatenating both face and pose landmarks
            row = pose_row+face_row

            # #adding class name to the row
            # row.insert(0, class_name)

            # with open('coords.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)

            #make predictions
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)

            #grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [1920,1080]).astype(int))
            
            cv2.rectangle(image, (coords[0], coords[1]+10), (coords[0]+len(body_language_class)*30, coords[1]-30), (245, 117, 16), -1)

            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            

            #get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1) #top corner, bottom corner, colour, thickness

            #display class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #display probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            

        except:
            pass

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
